In [1]:
import numpy as np

In [20]:
cond_channels = 256
cond_layers = 2
if cond_layers:
    dimensions = [64,]+[cond_channels]*(cond_layers-1)+[96,]
    in_dims = dimensions[:-1]
    out_dims = dimensions[1:]
    print(in_dims, out_dims, "\n")
    for i in range(len(in_dims)):
        print(in_dims[i], out_dims[i])
else:
    print("None")

[64, 256] [256, 96] 

64 256
256 96


**WaveGlow**

In [25]:
def MAC_calc(duration, n_audio_channel_init, L, kernal_size, C_wn, C_wn_middle, n_flows, n_layers, n_early_output, n_early_output_interval, C_mel):
    n_audio_channels = []
    n_audio = n_audio_channel_init
    for i in range(n_flows):
      if i % n_early_output_interval == 0 and i > 0:
        n_audio -= n_early_output
      n_audio_channels.append(n_audio) # audio channel after early output

    # in_layers
    WN_in_layers = L * kernal_size * C_wn * C_wn_middle * n_layers * n_flows
    print('MACs of in_layers', WN_in_layers / duration / 1e9)
    # cond layers
    WN_cond_layers = L * C_mel * C_wn_middle * n_layers * n_flows
    print('MACs of cond_layers', WN_cond_layers / duration / 1e9)
    # res skip layers
    WN_res_layers = (L * C_wn * C_wn_middle * (n_layers - 1) + L * C_wn * C_wn) * n_flows
    print('MACs of res_skip_layers', WN_res_layers / duration / 1e9)
    # invertible convs
    inv1x1 = np.sum([n**2 * L for n in n_audio_channels])
    print('MACs of invertible conv layers', inv1x1 / duration / 1e9)
    # start
    starts = np.sum([n / 2 * C_wn * L for n in n_audio_channels])
    print('MACs of start conv layers', starts / duration / 1e9)
    # end
    ends = np.sum([C_wn * n * L for n in n_audio_channels])
    print('MACs of end conv layers', ends / duration / 1e9)
    # total
    WG_total = WN_in_layers + WN_cond_layers + WN_res_layers + inv1x1 + starts + ends
    print('Total number of MACs is', WG_total / duration / 1e9)

In [26]:
duration = 16384/22050
n_audio_channel_init = 8 # initial audio channel
L = (22050*duration)//n_audio_channel_init # audio length
kernal_size = 3
C_wn = 256 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 12
n_layers = 8
n_early_output = 2
n_early_output_interval = 4
C_mel = 80 * n_layers # After upsampling and unfolding 

MAC_calc(duration, n_audio_channel_init, L, kernal_size, C_wn, C_wn_middle, n_flows, n_layers, n_early_output, n_early_output_interval, C_mel)

MACs of in_layers 104.0449536
MACs of cond_layers 86.704128
MACs of res_skip_layers 32.514048
MACs of invertible conv layers 0.0012789
MACs of start conv layers 0.0254016
MACs of end conv layers 0.0508032
Total number of MACs is 223.3406133


In [45]:
duration = 16384/22050
n_audio_channel_init = 24 # initial audio channel
L = (22050*duration)//n_audio_channel_init # audio length
kernal_size = 3
C_wn = 256 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 24
n_layers = 9
n_early_output = 2
n_early_output_interval = 24
C_mel = 160 # After upsampling and unfolding 

MAC_calc(duration, n_audio_channel_init, L, kernal_size, C_wn, C_wn_middle, n_flows, n_layers, n_early_output, n_early_output_interval, C_mel)

MACs of in_layers 77.9575104
MACs of cond_layers 16.241148
MACs of res_skip_layers 24.5421792
MACs of invertible conv layers 0.012688396875
MACs of start conv layers 0.06767145
MACs of end conv layers 0.1353429
Total number of MACs is 118.956540346875


In [46]:
duration = 16384/22050
n_audio_channel_init = 120 # initial audio channel
L = (22050*duration)//n_audio_channel_init # audio length
kernal_size = 3
C_wn = 384 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 16
n_layers = 5
n_early_output = 2
n_early_output_interval = 24
C_mel = 160 # After upsampling and unfolding 

MAC_calc(duration, n_audio_channel_init, L, kernal_size, C_wn, C_wn_middle, n_flows, n_layers, n_early_output, n_early_output_interval, C_mel)

MACs of in_layers 12.954816
MACs of cond_layers 1.79928
MACs of res_skip_layers 3.8864448
MACs of invertible conv layers 0.042170625
MACs of start conv layers 0.067473
MACs of end conv layers 0.134946
Total number of MACs is 18.885130425


In [ ]:
#					Length	n_chann	compute	reduc	MOS
WaveGlow    		2048	8		228.9	1x		4.57±0.04
SqueezeWave-128L	128		256		3.78	60x		4.07±0.06
SqueezeWave-64L		64		256		2.16	106x	3.77±0.05
SqueezeWave-128S	128		128		1.06	214x	3.79±0.05
SqueezeWave-64S		64		128		0.68	332x	2.74±0.04

In [75]:
print(16384/128, 16384/64)

128.0 256.0


In [15]:
L = 2048 # audio length
n_audio_channel_init = 8 # initial audio channel 
C_mel = 80 * 8 # After upsampling and unfolding 
kernal_size = 3
C_wn = 256 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 12
n_layers = 8
n_early_output = 2
n_early_output_interval = 4
duration = 0.725

n_audio_channels = []
n_audio = n_audio_channel_init
for i in range(n_flows):
  if i % n_early_output_interval == 0 and i > 0:
    n_audio -= n_early_output
  n_audio_channels.append(n_audio) # audio channel after early output

# in_layers
WN_in_layers = L * kernal_size * C_wn * C_wn_middle * n_layers * n_flows
print('MACs of in_layers', WN_in_layers / duration / 1e9)
# cond layers
WN_cond_layers = L * C_mel * C_wn_middle * n_layers * n_flows 
print('MACs of cond_layers', WN_cond_layers / duration / 1e9)
# res skip layers
WN_res_layers = (L * C_wn * C_wn_middle * (n_layers - 1) + L * C_wn * C_wn) * n_flows
print('MACs of res_skip_layers', WN_res_layers / duration / 1e9)
# invertible convs
inv1x1 = np.sum([n**2 * L for n in n_audio_channels])
print('MACs of invertible conv layers', inv1x1 / duration / 1e9)
# start
starts = np.sum([n / 2 * C_wn * L for n in n_audio_channels])
print('MACs of start conv layers', starts / duration / 1e9)
# end
ends = np.sum([C_wn * n * L for n in n_audio_channels])
print('MACs of end conv layers', ends / duration / 1e9)
# total
WG_total = WN_in_layers + WN_cond_layers + WN_res_layers + inv1x1 + starts + ends
print('Total number of MACs is', WG_total / duration / 1e9)

MACs of in_layers 106.63367079724138
MACs of cond_layers 88.86139233103448
MACs of res_skip_layers 33.32302212413793
MACs of invertible conv layers 0.00131072
MACs of start conv layers 0.02603361103448276
MACs of end conv layers 0.05206722206896552
Total number of MACs is 228.89749680551725


SqueezeWave L=64, C=128

In [6]:
L = 64 # audio length
n_audio_channel_init = 256 # initial audio channel 
L_mel = 64 # mel-spectrogram length
C_mel =80 # mel-spectrogram channel 
kernal_size = 3
C_wn = 128 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 12
n_layers = 8
n_early_output = 16
n_early_output_interval = 2
duration = 0.725

n_audio_channels = []
n_audio = n_audio_channel_init
for i in range(n_flows):
  if i % n_early_output_interval == 0 and i > 0:
    n_audio -= n_early_output
  n_audio_channels.append(n_audio) # audio channel after early output

# in_layers
WN_in_layers = L * kernal_size * C_wn * n_layers * n_flows # depthwise
WN_in_layers += L * C_wn * C_wn_middle * n_layers * n_flows # pointwise
print('MACs of in_layers', WN_in_layers / duration / 1e9)
# cond_layers
WN_cond_layers = L_mel * C_mel * C_wn_middle * n_layers * n_flows
print('MACs of cond_layers', WN_cond_layers / duration / 1e9)
# res_skip_layers
WN_res_layers = L * C_wn * C_wn * n_layers * n_flows
print('MACs of res_skip_layers', WN_res_layers / duration / 1e9)
# invertible convs
inv1x1 = np.sum([n**2 * L for n in n_audio_channels])
print('MACs of invertible conv layers', inv1x1 / duration / 1e9)
# start
starts = np.sum([n / 2 * C_wn * L for n in n_audio_channels])
print('MACs of start conv layers', starts / duration / 1e9)
#end
ends = np.sum([C_wn * n * L for n in n_audio_channels])
print('MACs of end conv layers', ends / duration / 1e9)
# total
total = WN_in_layers + WN_cond_layers + WN_res_layers + inv1x1 + starts + ends
print('Total number of MACs is', total / duration / 1e9)
print('Reduction compared with WaveGlow', WG_total / total)

MACs of in_layers 0.2809460524137931
MACs of cond_layers 0.17355740689655172
MACs of res_skip_layers 0.1388459255172414
MACs of invertible conv layers 0.0502141351724138
MACs of start conv layers 0.014643906206896554
MACs of end conv layers 0.029287812413793107
Total number of MACs is 0.6874952386206896
Reduction compared with WaveGlow 332.9441193872855


**SqueezeWave L=64, C=256**

In [4]:
L = 64 # audio length
n_audio_channel_init = 256 # initial audio channel 
L_mel = 64 # mel-spectrogram length
C_mel =80 # mel-spectrogram channel 
kernal_size = 3
C_wn = 256 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 12
n_layers = 8
n_early_output = 16
n_early_output_interval = 2
duration = 0.725

n_audio_channels = []
n_audio = n_audio_channel_init
for i in range(n_flows):
  if i % n_early_output_interval == 0 and i > 0:
    n_audio -= n_early_output
  n_audio_channels.append(n_audio) # audio channel after early output

# in_layers
WN_in_layers = L * kernal_size * C_wn * n_layers * n_flows # depthwise
WN_in_layers += L * C_wn * C_wn_middle * n_layers * n_flows # pointwise
print('MACs of in_layers', WN_in_layers / duration / 1e9)
# cond_layers
WN_cond_layers = L_mel * C_mel * C_wn_middle * n_layers * n_flows
print('MACs of cond_layers', WN_cond_layers / duration / 1e9)
# res_skip_layers
WN_res_layers = L * C_wn * C_wn * n_layers * n_flows
print('MACs of res_skip_layers', WN_res_layers / duration / 1e9)
# invertible convs
inv1x1 = np.sum([n**2 * L for n in n_audio_channels])
print('MACs of invertible conv layers', inv1x1 / duration / 1e9)
# start
starts = np.sum([n / 2 * C_wn * L for n in n_audio_channels])
print('MACs of start conv layers', starts / duration / 1e9)
#end
ends = np.sum([C_wn * n * L for n in n_audio_channels])
print('MACs of end conv layers', ends / duration / 1e9)
# total
total = WN_in_layers + WN_cond_layers + WN_res_layers + inv1x1 + starts + ends
print('Total number of MACs is', total / duration / 1e9)
print('Reduction compared with WaveGlow', WG_total / total)

MACs of in_layers 1.1172758068965518
MACs of cond_layers 0.34711481379310344
MACs of res_skip_layers 0.5553837020689656
MACs of invertible conv layers 0.0502141351724138
MACs of start conv layers 0.029287812413793107
MACs of end conv layers 0.058575624827586215
Total number of MACs is 2.157851895172414
Reduction compared with WaveGlow 106.07655572544667


**SqueezeWave L=128, C=128**

In [ ]:
L = 128 # audio length
n_audio_channel_init = 128 # initial audio channel 
L_mel = 64 # mel-spectrogram length
C_mel =80 # mel-spectrogram channel 
kernal_size = 3
C_wn = 128 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 12
n_layers = 8
n_early_output = 16
n_early_output_interval = 2
duration = 0.725

n_audio_channels = []
n_audio = n_audio_channel_init
for i in range(n_flows):
  if i % n_early_output_interval == 0 and i > 0:
    n_audio -= n_early_output
  n_audio_channels.append(n_audio) # audio channel after early output

# in_layers
WN_in_layers = L * kernal_size * C_wn * n_layers * n_flows # depthwise
WN_in_layers += L * C_wn * C_wn_middle * n_layers * n_flows # pointwise
print('MACs of in_layers', WN_in_layers / duration / 1e9)
# cond_layers
WN_cond_layers = L_mel * C_mel * C_wn_middle * n_layers * n_flows
print('MACs of cond_layers', WN_cond_layers / duration / 1e9)
# res_skip_layers
WN_res_layers = L * C_wn * C_wn * n_layers * n_flows
print('MACs of res_skip_layers', WN_res_layers / duration / 1e9)
# invertible convs
inv1x1 = np.sum([n**2 * L for n in n_audio_channels])
print('MACs of invertible conv layers', inv1x1 / duration / 1e9)
# start
starts = np.sum([n / 2 * C_wn * L for n in n_audio_channels])
print('MACs of start conv layers', starts / duration / 1e9)
#end
ends = np.sum([C_wn * n * L for n in n_audio_channels])
print('MACs of end conv layers', ends / duration / 1e9)
# total
total = WN_in_layers + WN_cond_layers + WN_res_layers + inv1x1 + starts + ends
print('Total number of MACs is', total / duration / 1e9)
print('Reduction compared with WaveGlow', WG_total / total)

('MACs of in_layers', 0.5618921048275862)
('MACs of cond_layers', 0.17355740689655172)
('MACs of res_skip_layers', 0.2776918510344828)
('MACs of invertible conv layers', 0.017988502068965517)
('MACs of start conv layers', 0.011932071724137933)
('MACs of end conv layers', 0.023864143448275865)
('Total number of MACs is', 1.06692608)
('Reduction compared with WaveGlow', 214)


**SqueezeWave L=128, C=256**

In [ ]:
L = 128 # audio length
n_audio_channel_init = 128 # initial audio channel 
L_mel = 64 # mel-spectrogram length
C_mel =80 # mel-spectrogram channel 
kernal_size = 3
C_wn = 256 # input channel size of in_layer
C_wn_middle = C_wn * 2 # output channel size of in_layer and cond_layer
n_flows = 12
n_layers = 8
n_early_output = 16
n_early_output_interval = 2
duration = 0.725

n_audio_channels = []
n_audio = n_audio_channel_init
for i in range(n_flows):
  if i % n_early_output_interval == 0 and i > 0:
    n_audio -= n_early_output
  n_audio_channels.append(n_audio) # audio channel after early output

# in_layers
WN_in_layers = L * kernal_size * C_wn * n_layers * n_flows # depthwise
WN_in_layers += L * C_wn * C_wn_middle * n_layers * n_flows # pointwise
print('MACs of in_layers', WN_in_layers / duration / 1e9)
# cond_layers
WN_cond_layers = L_mel * C_mel * C_wn_middle * n_layers * n_flows
print('MACs of cond_layers', WN_cond_layers / duration / 1e9)
# res_skip_layers
WN_res_layers = L * C_wn * C_wn * n_layers * n_flows
print('MACs of res_skip_layers', WN_res_layers / duration / 1e9)
# invertible convs
inv1x1 = np.sum([n**2 * L for n in n_audio_channels])
print('MACs of invertible conv layers', inv1x1 / duration / 1e9)
# start
starts = np.sum([n / 2 * C_wn * L for n in n_audio_channels])
print('MACs of start conv layers', starts / duration / 1e9)
#end
ends = np.sum([C_wn * n * L for n in n_audio_channels])
print('MACs of end conv layers', ends / duration / 1e9)
# total
total = WN_in_layers + WN_cond_layers + WN_res_layers + inv1x1 + starts + ends
print('Total number of MACs is', total / duration / 1e9)
print('Reduction compared with WaveGlow', WG_total / total)

('MACs of in_layers', 2.2345516137931036)
('MACs of cond_layers', 0.34711481379310344)
('MACs of res_skip_layers', 1.1107674041379312)
('MACs of invertible conv layers', 0.017988502068965517)
('MACs of start conv layers', 0.023864143448275865)
('MACs of end conv layers', 0.04772828689655173)
('Total number of MACs is', 3.7820147641379314)
('Reduction compared with WaveGlow', 60)
